import the libraries

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize




from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.layers import Dense, SimpleRNN,LSTM,Bidirectional,GRU,Dropout, TextVectorization, Embedding, Input
from keras.regularizers import l2
import tensorflow as tf


import the data set

In [2]:


data = pd.read_csv("/content/tweet_emotions.csv")

In [3]:
pd.set_option('display.max_colwidth', None)
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part =[
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin on your call...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,"@dannycastillo We want to trade with someone who has Houston tickets, but no one will."


Data preprocessing

In [4]:
data.shape

(40000, 3)

In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [6]:
data.isna().sum()

,0
tweet_id,0
sentiment,0
content,0


In [7]:
#since the data type is object convert it into string
data = data.astype('string')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  string
 1   sentiment  40000 non-null  string
 2   content    40000 non-null  string
dtypes: string(3)
memory usage: 937.6 KB


In [8]:
data['sentiment'].value_counts()

,count
sentiment,
neutral,8638
worry,8459
happiness,5209
sadness,5165
love,3842
surprise,2187
fun,1776
relief,1526
hate,1323


In [9]:
classes= pd.get_dummies(data['sentiment'],dtype='int')
classes

,anger,boredom,empty,enthusiasm,fun,happiness,hate,love,neutral,relief,sadness,surprise,worry
0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0,0,0,0,0,0,0,0,1,0,0,0,0
39996,0,0,0,0,0,0,0,1,0,0,0,0,0
39997,0,0,0,0,0,0,0,1,0,0,0,0,0
39998,0,0,0,0,0,1,0,0,0,0,0,0,0


Data Cleaning

In [10]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [11]:

def clean_text(text):


    # the column 'content' have some special charters so remove it.
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join tokens back into a single string
    return ' '.join(tokens)


In [22]:
data['content'] = data['content'].apply(clean_text)

# Display the cleaned dataset
data.head()


,tweet_id,sentiment,content
0,1956967341,empty,tiffanylue know listenin bad habit earlier started freakin part
1,1956967666,sadness,layin n bed headache ughhhhwaitin call
2,1956967696,sadness,funeral ceremonygloomy friday
3,1956967789,enthusiasm,want hang friend soon
4,1956968416,neutral,dannycastillo want trade someone houston ticket one


Train Test Split

In [15]:
x_train , x_test, y_train, y_test = train_test_split(data['sentiment'],classes,test_size = 0.1 , random_state =0 , stratify=classes )

In [16]:
x_test

,sentiment
31231,worry
39583,happiness
10421,neutral
31056,surprise
3624,worry
...,...
9945,neutral
2337,surprise
9687,enthusiasm
21644,hate


In [17]:
#Text Vectorization
vectorizer = TextVectorization(max_tokens=5000)
vectorizer.adapt(x_train)





LSTM

In [18]:
model = Sequential()

model.add(vectorizer)
model.add(Embedding(input_dim=5000, output_dim=256,mask_zero=True))

model.add(LSTM(256,activation='tanh',return_sequences = True, kernel_regularizer=l2(0.02)))
model.add(LSTM(256,activation='tanh',return_sequences = True, kernel_regularizer=l2(0.02)))
model.add(LSTM(64, return_sequences = True, kernel_regularizer=l2(0.02)))
model.add(LSTM(32))


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(4,activation='softmax'))

#Compiling model_1
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [19]:
history = model.fit(x_train,y_train,
                    epochs = 10,
                    batch_size = 32,
                    validation_data = (x_test,y_test))

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(32, 13), output.shape=(None, 4)